In [1]:
import os
import boto3
import pandas as pd

In [3]:
# Initialize the S3 client
s3_client = boto3.client('s3')

# Bucket name and file paths
bucket_name = 'ca-climate-index'
directory = '0_map_data/crosswalk_data/CCI_Projects_Project_Category_Update_02142024.xlsm'

print('Pulling file')
s3_client.download_file(bucket_name, directory, 'CCI_Projects_Project_Category_Update_02142024.xlsm')
print('File pulled')

Pulling file
File pulled


In [4]:
crosswalk_data = pd.read_excel('CCI_Projects_Project_Category_Update_02142024.xlsm')

In [13]:
display(crosswalk_data.columns.tolist())

['Project IDNumber',
 'Reporting Cycle Name',
 'Agency Name',
 'Program Name',
 'Program Description',
 'Sub Program Name',
 'Record Type',
 'Project Name',
 'Project Type',
 'Project Description',
 'SECTOR',
 'CATEGORY',
 'ACTION',
 'Census Tract',
 'Address',
 'Lat Long',
 'Senate\nDistrict',
 'Assembly\nDistrict',
 'County',
 'Total Project Cost',
 'Total Program GGRFFunding',
 'Project Life Years',
 'Total Project GHGReductions',
 'Annual Project GHGReductions',
 'Project Count',
 'Fiscal Year Funding Project',
 'Is Benefit Disadvantaged Communities',
 'Disadvantaged Community Criteria',
 'Disadvantaged Community Need',
 'Disadvantaged Community Census Tracts',
 'Total GGRFDisadvantaged Community Funding',
 'Disadvantaged Community Benefits Description',
 'Funding Benefiting Disadvantaged Communities',
 'Estimated Num Vehicles In Service',
 'Funding Within Disadvantage Communities',
 'Other Project Benefits Description',
 'VMTReductions',
 'Number Of Housing Units',
 'Number Of Aff

In [6]:
relevant_columns = [
    'Program Name',
    'Program Description',
    'Sub Program Name',
    'Project Type',
    'Project Description',
    'SECTOR',
    'CATEGORY',
    'ACTION',
    'Census Tract',
    'Total Project GHGReductions',
    'Project Count',
    'Is Benefit Disadvantaged Communities',
    'Estimated Number Of Trees To Be Planted',
    'Estimated Acres Restored',
    'Estimated Acres Treated',
]

In [7]:
data_of_interest = crosswalk_data[relevant_columns]

In [8]:
# Set display options to show all columns and rows
# pd.set_option('display.max_columns', None)  # To display all columns
# pd.set_option('display.max_rows', None)     # To display all rows

# Now display data_of_interest
display(data_of_interest)

,Program Name,Program Description,Sub Program Name,Project Type,Project Description,SECTOR,CATEGORY,ACTION,Census Tract,Total Project GHGReductions,Project Count,Is Benefit Disadvantaged Communities,Estimated Number Of Trees To Be Planted,Estimated Acres Restored,Estimated Acres Treated
0,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,50.0,7.0,0.0,0.0,0.0,0.0
1,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,41.0,7.0,0.0,0.0,0.0,0.0
2,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,97.0,14.0,0.0,0.0,0.0,0.0
3,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,68.0,10.0,1.0,0.0,0.0,0.0
4,Low Carbon Transportation,Provides mobile source incentives to reduce GH...,Clean Vehicle Rebate Project,NaN,CVRP promotes clean vehicle adoption in Califo...,"Zero-Emission Vehicles, Equipment, and Infrast...",Light-Duty Vehicles,Consumer Incentives,6.001400e+09,29.0,4.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133693,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,NaN,0.0,1.0,0.0,0.0,0.0,0.0
133694,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC improves expertise on climate change issu...,Climate Adaptation and Resilience,Sea Level Rise,Capacity-Building,NaN,0.0,1.0,0.0,0.0,0.0,0.0
133695,Climate Resilience Planning,Works with local governments on adaptation pla...,Climate Change Adaptation and Coastal Resilien...,State Operations,BCDC's Regulatory staff conducts comprehensive...,Climate Adaptation and Resilience,Wetland Restoration and Management,Permitting,NaN,0.0,1.0,0.0,0.0,0.0,0.0
133696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
climate_risk_dict = {
    'wildfire mitigation': ['wildfire', 'Wildfire', 'burn', 'controlled burns', 'firefighting', 'forest', 'reforest', 'reforestation', 'fire', 'vegetation management', 'roadside brushing', 'fuel break'],
    'sea level rise mitigation': ['sea level rise', 'slr', 'erosion', 'seawall', 'riparian', 'seawalls', 'shoreline', 'wetland', 'mangrove'],
    'extreme heat mitigation': ['heat', 'extreme heat', 'shade', 'shading', 'cooling center', 'cooling centers', 'heat-resistant', 'heat resistant', 'heat reducing', 'heat-reducing'],
    'drought mitigation': ['drought', 'dry', 'irrigation', 'soil moisture', 'rainwater harvest', 'rainwater harvesting', 'water storage', 'water allocation', 'water management'],
    'inland flooding mitigation': ['flood', 'flooding', 'inland flood', 'inland flooding', 'floodplain', 'floodproof', 'floodproofing', 'elevated flood', 'flood barrier', 'flood barriers', 'drainage'],
    'greenhouse gas mitigation': ['ghg', 'GHG', 'greenhouse gas', 'emission', 'emissions', 'carbon sequestration', 'electrification', 'carbon capture', 'solar power', 'wind energy', 'hydroelectricity', 'geothermal energy', 'biomass energy', 'Energy-efficiency']
}


In [11]:
# Function to check for keywords and return matched categories
def find_categories(description):
    flagged_categories = []
    for category, keywords in climate_risk_dict.items():
        for keyword in keywords:
            if keyword.lower() in description.lower():
                flagged_categories.append(category)
                break  # If a keyword is found, break to avoid duplicate category entry
    return ','.join(flagged_categories)

# Replace NaN values in 'ProjectDescription' column with 'missing'
data_of_interest['Project Description'].fillna('missing', inplace=True)

# Extract unique project descriptions
unique_descriptions = data_of_interest['Project Description'].unique()

# Create an empty DataFrame to accumulate results
results_df = pd.DataFrame(columns=['Project Type', 'Project Count', 'Project Description'])

# Filter cci_project_data based on unique_descriptions
filtered_data = data_of_interest[data_of_interest['Project Description'].isin(unique_descriptions)]

total_rows_processed = len(unique_descriptions)
total_rows_flagged = 0
entries_counter = {category: 0 for category in climate_risk_dict}
multiple_keys_count = 0
other_count = 0

for description in unique_descriptions:
    flagged_categories = find_categories(description)
    
    if flagged_categories:
        categories_list = flagged_categories.split(',')
        total_rows_flagged += 1
        
        if len(categories_list) > 1:
            multiple_keys_count += 1
        
        for category in categories_list:
            entries_counter[category] += 1
            
        # Find the rows that match the current description and extract other relevant columns
        matched_rows = filtered_data[filtered_data['Project Description'] == description][['Project Description']]
        
        # Add all flagged categories to the matched rows
        matched_rows['Flagged keywords'] = ','.join(categories_list)
        
        # Append the matched rows to the results DataFrame
        results_df = pd.concat([results_df, matched_rows], ignore_index=True)
    else:
        other_count += 1

# Exporting data to a new CSV file
results_df.drop_duplicates(inplace=True)  # Remove duplicates
results_df.to_csv('flagged_keywords_results.csv', index=False)

# Print counts
print(f"Total Rows Processed (unique descriptions): {total_rows_processed}")
print(f"Total Rows Flagged (caught from dictionary): {total_rows_flagged}")
for category, count in entries_counter.items():
    print(f"{category}: {count}")
print(f"Other: {other_count}")
print(f"Multiple Keys Found: {multiple_keys_count}")

C:\Users\jesse\AppData\Local\Temp\ipykernel_13648\2778882567.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_of_interest['Project Description'].fillna('missing', inplace=True)


Total Rows Processed (unique descriptions): 5236
Total Rows Flagged (caught from dictionary): 2182
wildfire mitigation: 951
sea level rise mitigation: 145
extreme heat mitigation: 173
drought mitigation: 727
inland flooding mitigation: 173
greenhouse gas mitigation: 846
Other: 3054
Multiple Keys Found: 738


In [ ]:
# Counting unique entries in 'IP_Project Category'
unique_categories = data_of_interest['Program Name'].nunique()

total_unique_project_desc = data_of_interest['Project Description'].nunique()

# Creating an empty dictionary to store counts for each unique 'IP_Project Category'
category_counts = {}

# Counting occurrences of each unique 'IP_Project Category' in the DataFrame
category_occurrences = data_of_interest['Program Name'].value_counts()

# Looping through each unique 'IP_Project Category'
for category in data_of_interest['Program Name'].unique():
    # Counting unique 'ProgramDescription' for each 'IP_Project Category'
    unique_program_desc = data_of_interest[data_of_interest['IP_Project Category'] == category]['Program Description'].nunique()
    # Counting unique 'ProjectDescription' for each 'IP_Project Category'
    unique_project_desc = data_of_interest[data_of_interest['IP_Project Category'] == category]['Project Description'].nunique()
    # Storing the counts in the dictionary
    category_counts[category] = {'Unique Program Description': unique_program_desc, 'Unique Project Description': unique_project_desc}
    
    # Creating an empty dictionary to store counts for each unique 'IP_Project Subcategory' within the category
    subcategory_counts = {}
    
    # Looping through each unique 'IP_Project Subcategory' within the 'IP_Project Category'
    for subcategory in data_of_interest[data_of_interest['IP_Project Category'] == category]['IP_Project Subcategory'].unique():
        # Counting occurrences of each unique 'IP_Project Subcategory' within the category
        subcategory_count = data_of_interest[(data_of_interest['IP_Project Category'] == category) & (data_of_interest['IP_Project Subcategory'] == subcategory)].shape[0]
        # Storing the count in the subcategory dictionary
        subcategory_counts[subcategory] = subcategory_count
    
    # Storing the subcategory counts for each category
    category_counts[category]['Subcategory Counts'] = subcategory_counts

# Writing the results to a text file
output_file = 'cci_project_summary.txt'

# Assuming category_occurrences is a dictionary containing your data
sorted_categories = sorted(category_occurrences.items(), key=lambda x: x[1], reverse=True)

# To just give IP Project occurances and number of each
'''
with open("your_file.txt", "w") as file:
    for category, count in sorted_categories:
        file.write(f"  - Number of IP Project '{category}' in dataset: {count}\n")
'''
# To give comprehensive summary stats
with open(output_file, 'w') as file:
    file.write(f"Number of unique 'IP_Project Category': {unique_categories}\n")
    file.write(f"Total number of unique 'ProjectDescription': {total_unique_project_desc}\n")
    file.write("Category Counts:\n")
    for category, counts in category_counts.items():
        file.write(f"- Category: {category}\n")
        file.write(f"  - Number of IP Project '{category}' in dataset: {category_occurrences[category]}\n")
        file.write(f"  - Number of unique Program descriptions: {counts['Unique ProgramDescription']}\n")
        file.write(f"  - Number of unique Project descriptions: {counts['Unique ProjectDescription']}\n")
        file.write("  - Subcategory Counts:\n")
        for subcategory, count in counts['Subcategory Counts'].items():
            file.write(f"    - {subcategory}: {count}\n")

print(f"Results saved to {output_file}")